In [1]:
import numpy as np 
import pandas as pd 
import os 
import datetime
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, TimeoutError, Lock, Value, current_process, cpu_count
from functools import partial

import data_preprocess_function as func

In [2]:
filepath = "D:\\Strategic_Trading\\2330\\trade\\"
file_list = os.listdir(filepath)
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.ReadFile_2330, filepath=filepath), file_list), total=len(file_list)), 1):
            df_list.append(x)
            
data = pd.concat(df_list)

In [3]:
data

,ts,Trade Price,Trade Volume(share)
1,2013-07-10 09:00:06,104.5,1000
2,2013-07-10 09:00:06,104.5,7000
4,2013-07-10 09:00:06,104.5,5000
7,2013-07-10 09:00:06,104.5,1000
9,2013-07-10 09:00:06,104.5,1000
...,...,...,...
29234,2018-06-25 13:30:00,218.0,2000
29236,2018-06-25 13:30:00,218.0,1000
29238,2018-06-25 13:30:00,218.0,1000
29240,2018-06-25 13:30:00,218.0,1000


In [4]:
df_list = [group[1] for group in data.groupby(data["ts"].dt.date)]
OHLCV_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.OHLCV_2330, frequency=1), df_list), total=len(df_list)), 1):
                OHLCV_list.append(x)

df = pd.concat(OHLCV_list)

In [6]:
df = df.sort_values(by=['ts'])
df = df.reset_index(drop=True)
df

,ts,open,high,low,close,vol,VWAP
0,2013-07-01 09:00:00,109.0,109.0,109.0,109.0,2783000,109.000000
1,2013-07-01 09:01:00,109.0,109.0,108.5,108.5,512000,108.879883
2,2013-07-01 09:02:00,108.5,108.5,108.5,108.5,361000,108.500000
3,2013-07-01 09:03:00,108.5,108.5,108.5,108.5,463000,108.500000
4,2013-07-01 09:04:00,109.0,109.5,109.0,109.5,589000,109.064516
...,...,...,...,...,...,...,...
324799,2018-06-29 13:21:00,216.0,216.5,216.0,216.0,22000,216.045455
324800,2018-06-29 13:22:00,216.0,216.0,216.0,216.0,10000,216.000000
324801,2018-06-29 13:23:00,216.0,216.0,216.0,216.0,88000,216.000000
324802,2018-06-29 13:24:00,216.0,216.0,216.0,216.0,102000,216.000000


In [7]:
df_list = [group[1] for group in df.groupby(df["ts"].dt.date)]
timelist = df["ts"].dt.time.unique().tolist()
fill_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.FillMissingTime, timelist=timelist), df_list), total=len(df_list)), 1):
                fill_list.append(x)

df = pd.concat(fill_list)

In [9]:
df = df.sort_values(by=['ts'])
df = df.reset_index(drop=True)
df

,ts,open,high,low,close,vol,VWAP
0,2013-07-01 09:00:00,109.0,109.0,109.0,109.0,2783000.0,109.000000
1,2013-07-01 09:01:00,109.0,109.0,108.5,108.5,512000.0,108.879883
2,2013-07-01 09:02:00,108.5,108.5,108.5,108.5,361000.0,108.500000
3,2013-07-01 09:03:00,108.5,108.5,108.5,108.5,463000.0,108.500000
4,2013-07-01 09:04:00,109.0,109.5,109.0,109.5,589000.0,109.064516
...,...,...,...,...,...,...,...
326909,2018-06-29 13:21:00,216.0,216.5,216.0,216.0,22000.0,216.045455
326910,2018-06-29 13:22:00,216.0,216.0,216.0,216.0,10000.0,216.000000
326911,2018-06-29 13:23:00,216.0,216.0,216.0,216.0,88000.0,216.000000
326912,2018-06-29 13:24:00,216.0,216.0,216.0,216.0,102000.0,216.000000


In [21]:
df.to_csv("D:\\Strategic_Trading\\PriceForecast\\data\\2330\\min_OHLCV_new.csv", index=False)